Код к соревнованию https://www.kaggle.com/competitions/nlp-getting-started/  
Задача - научить модель распознавать твиты, в которых есть информация о природных катаклизмах, а в каких нет. Датасет состоит из 10000 твитов, размеченных вручную.

In [1]:
import numpy as np
import pandas as pd
from string import punctuation
punctuation += "«»—…“”"

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model, model_selection, preprocessing

Импортируем датасеты чтобы обучать модель и проверять качество:

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
print(train_df.shape, test_df.shape)

(7613, 5) (3263, 4)


Посмотрим на примеры:

In [3]:
# нейтральный твит
train_df[train_df["target"] == 0]["text"].values[1]

'I love fruits'

In [4]:
# твит про катаклизм
train_df[train_df["target"] == 1]["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

Проведём предобработку текстов: уберём пунктуацию и приведём слова к первоначальной форме.

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()

In [6]:
# вспомогательная функция для лемматизатора
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [7]:
def preprocess(sentence):
    return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)).lower() for w in nltk.word_tokenize(sentence)\
            if w not in punctuation])

In [8]:
preprocessed_train = [preprocess(sentence) for sentence in train_df["text"]]
preprocessed_test = [preprocess(sentence) for sentence in test_df["text"]]

Векторизуем тексты:

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [10]:
train_vectors = tfidf_vectorizer.fit_transform(pd.Series(preprocessed_train))
test_vectors = tfidf_vectorizer.transform(pd.Series(preprocessed_test))

Инициализируем классификатор:

In [11]:
clf = linear_model.RidgeClassifier()

Проверяем качество полученной модели:

In [12]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.6401742 , 0.61984733, 0.67342799])

Я пробовала ещё два классификатора, но они показали результаты хуже.

In [13]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier()

In [14]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.53325198, 0.48230088, 0.57446809])

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()

In [16]:
scores = model_selection.cross_val_score(clf, train_vectors, train_df["target"], cv=3, scoring="f1")
scores

array([0.51275669, 0.52537646, 0.56526549])

Подготавливаем файл для соревнования:

In [ ]:
clf = linear_model.RidgeClassifier()

In [23]:
clf.fit(train_vectors, train_df["target"])

RidgeClassifier()

In [24]:
sample_submission = pd.read_csv("sample_submission.csv")

In [25]:
sample_submission["target"] = clf.predict(test_vectors)

In [26]:
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [27]:
sample_submission.to_csv("submission.csv", index=False)